In [ ]:
%pip install ultralytics

import torch
from ultralytics import YOLO

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


In [ ]:
# Path to your data.yaml
data_path = '../data/data.yaml'

# Show the data.yaml content
with open(data_path, 'r') as f:
    print(f.read())


In [ ]:
model = YOLO('../models/yolov8n.pt')
results = model.train(
    data='../data/data.yaml',
    epochs=100,
    patience=10,
    imgsz=640,
    batch=8,
    name='wardrobe_yolov8n_macos',
    project='../outputs'
)

# Run validation on the best model
metrics = model.val()
print(metrics)


In [ ]:
%pip install opencv-python

import cv2
from ultralytics import YOLO

# Load your trained model
model = YOLO('../outputs/models/yolov8n_finetuned.pt')

# Open webcam 1 (change the index as needed)
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Run YOLO prediction
    results = model.predict(frame, show=True)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()